In [164]:
from __future__ import print_function

import os
import sys
from subprocess import Popen, check_output
from sys import stderr
from time import sleep
import io
import matplotlib.pyplot as plt
import pandas as pd
import tempfile
import subprocess
import psutil

In [177]:
def GetProcessAttributes(pids):
    processAttributes = []
    for pid in pids:
        proc = psutil.Process(pid)
        processAttributes.append({'pid':     pid,
                                  'name':    proc.name(),
                                  'exe':     proc.exe(),                  
                                  'cmdline': proc.cmdline(),
                                  'status':  proc.status()})
    processAttributes = pd.DataFrame(processAttributes)
    return processAttributes

In [182]:
def MonitorNvidiaGPU():
    'Function that monitors Running Processes on Nvidia GPU'
    '''
        Returns a DataFrame (pid, process_name, cmdline, used_gpu_memory, utilization)
    '''
    getGPUProcesses = 'nvidia-smi pmon -c 1 -s mu'
    
    proc = subprocess.Popen(getGPUProcesses, shell=True, stdout=subprocess.PIPE)
    output = proc.stdout.read().decode('utf-8').split('\n')
    
    # Remove the line describing the units of each feature
    del output[1]
    # convert to csv format...
    output[0] = output[0].replace('# ', '')
    output = [line.strip() for line in output]
    output = [','.join(line.split()) for line in output]
    # ...and drop the command feature (will be added later)...
    output = [','.join(line.split(',')[:8]) for line in output]
    # ...and convert to DataFrame
    procsGPU = pd.read_csv(io.StringIO('\n'.join(output)), header=0)
    procsGPUFeats = GetProcessAttributes(procsGPU.pid.values)
    
    return procsGPU.merge(procsGPUFeats, on='pid', how='inner')

In [184]:
%timeit MonitorNvidiaGPU()

36.1 ms ± 2.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Notes:
### * Decide if we use the default Nvidia thread or our own (So Far: our own)

In [77]:
#!/usr/bin/python
from subprocess import Popen, PIPE, STDOUT
import pty
import os

cmd = 'nvidia-smi pmon -c 1 -s mu'

proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
output = proc.stdout.read().decode('utf-8').split('\n')
output

['# gpu        pid  type    fb    sm   mem   enc   dec   command',
 '# Idx          #   C/G    MB     %     %     %     %   name',
 '    0       1107     G   331     0     0     0     0   Xorg           ',
 '    0       3917     G   135     0     2     0     0   compiz         ',
 '    0       4479     G    60     0     1     0     0   chrome --type=g',
 '    0       6473     G   230     0     0     0     0   1.0.80.480 --la',
 '']